# Neighborhoods in Toronto : Segment and Cluster
## Notebook for assignment for Week 3 of Applied Data science capstone
By - Abhinav Paul || Dated - 25/04/2020

## Part 1 (Step 1-4)- Data extraction and Dataframe creation

In [1]:
# library imports

import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim 
import requests 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from bs4 import BeautifulSoup

print('Libraries import - Complete')

Libraries import - Complete


Web scraping for data from Wikipage

In [2]:
# Data Web scraping
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
toronto_temp_data = pd.read_html(url)
toronto_temp_data = pd.DataFrame(toronto_temp_data[0])
toronto_temp_data.head(12)

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,NaN
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


Cleaning Data

In [3]:
# drop Borough = Not assigned
toronto_data = toronto_temp_data[toronto_temp_data['Borough']!='Not assigned']
toronto_data.reset_index(inplace=True,drop=True)
#toronto_data.drop(columns=['index'],inplace=True)

# checking for duplicate postal code in Neighborhoods
tor_grp = toronto_data.groupby(by=['Postal code']).count()
print('Toronto Postal codes summary ...\n')
print(toronto_data.count())
print('\nToronto Postal codes with multiple Borough and neighborhoods ...\n')
print(tor_grp[tor_grp['Neighborhood']>1].count())

# checking for NaN in Neighborhood
toronto_data['Neighborhood'].fillna(value=toronto_data['Borough']) 

print('\nSince duplicate were found no treat for the Neighborhood was done\n\nFinal Toronto Data shape looks like ...\n')
toronto_data.shape


Toronto Postal codes summary ...

Postal code     103
Borough         103
Neighborhood    103
dtype: int64

Toronto Postal codes with multiple Borough and neighborhoods ...

Borough         0
Neighborhood    0
dtype: int64

Since duplicate were found no treat for the Neighborhood was done

Final Toronto Data shape looks like ...



(103, 3)

## Part 2 - Merging Longitude and Latitude data for Toronto Neighborhood

In [4]:
# Geo data from csv file

path = 'http://cocl.us/Geospatial_data'
geo_data = pd.read_csv(path)
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
# Merging Neighborhood and Geo data 

toronto_geodata = pd.merge(toronto_data,geo_data, how = 'left', left_on ='Postal code', right_on = 'Postal Code')
toronto_geodata.drop(columns = ['Postal Code'],inplace=True)
toronto_geodata.head(12)

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## Part 3 - K-Means Cluster Analysis

In [6]:
#Selecting the data with Borough containg Trornto

Toronto_borough_data = toronto_geodata[toronto_geodata['Borough'].str.contains('Toronto')==True]
#Toronto_borough_data = toronto_geodata
Toronto_borough_data.reset_index(inplace=True,drop=True)
Toronto_borough_data.drop(columns=['Postal code'],inplace=True)

print('The dataframe has {} boroughs and {} neighborhoods.\n'.format(
        len(Toronto_borough_data['Borough'].unique()),
        Toronto_borough_data.shape[0]))
Toronto_borough_data.head()

The dataframe has 4 boroughs and 39 neighborhoods.



/Users/abhinavpaul/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
1,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,Downtown Toronto,St. James Town,43.651494,-79.375418
4,East Toronto,The Beaches,43.676357,-79.293031


Map generation for toronto with Neighborhoods highlighted

In [7]:
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.\n'.format(latitude, longitude))

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_borough_data['Latitude'],
                                           Toronto_borough_data['Longitude'],
                                           Toronto_borough_data['Borough'],
                                           Toronto_borough_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

/Users/abhinavpaul/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.



Foursquare Credentials and Version

In [8]:
CLIENT_ID = 'TBPLKSLPDCOMMX2YIDPJ5UG2NRWHG4YOB3VXMPJI5YYP54H3' # your Foursquare ID
CLIENT_SECRET = 'OV15HD3ZH2152BWBWDWFSKBEONQJNSQCJAI0S4BO0WLFWLPE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Let's explore Boroughs in our dataframe and top 150 venues in them within a radius of 500 meters

In [9]:
## borowing the getNearbyVenue function
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=150):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
 print('Neigbourhood Names:\n')
toronto_venues = getNearbyVenues(names=Toronto_borough_data['Neighborhood'],
                                   latitudes=Toronto_borough_data['Latitude'],
                                   longitudes=Toronto_borough_data['Longitude'])

Neigbourhood Names:

Regent Park / Harbourfront
Queen's Park / Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
The Danforth West / Riverdale
Toronto Dominion Centre / Design Exchange
Brockton / Parkdale Village / Exhibition Place
India Bazaar / The Beaches West
Commerce Court / Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park / The Junction South
North Toronto West
The Annex / North Midtown / Yorkville
Parkdale / Roncesvalles
Davisville
University of Toronto / Harbord
Runnymede / Swansea
Moore Park / Summerhill East
Kensington Market / Chinatown / Grange Park
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / 

EDA section

In [11]:
##EDA section
print('Shape of the data :')
print(toronto_venues.shape)
toronto_venues.head()

Shape of the data :
(1612, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park / Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Regent Park / Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Regent Park / Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,Regent Park / Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,Regent Park / Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


Neighborhood count

In [12]:
#Neighborhood count
toronto_venues.groupby('Neighborhood').count()

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

Print each neighborhood along with the top 5 most common venues

In [13]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.07
1    Cocktail Bar  0.05
2        Beer Bar  0.04
3  Farmers Market  0.04
4            Café  0.04


----Brockton / Parkdale Village / Exhibition Place----
                   venue  freq
0                   Café  0.14
1            Coffee Shop  0.09
2         Breakfast Spot  0.09
3  Performing Arts Venue  0.05
4              Pet Store  0.05


----Business reply mail Processing CentrE----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.06
2       Garden Center  0.06
3             Brewery  0.06
4          Skate Park  0.06


----CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport----
              venue  freq
0   Airport Service  0.17
1    Airport Lounge  0.11
2  Airport Terminal  0.11
3   Harbor / Marina  0.06
4           Airport  0.06


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.21
1   Italian

In [14]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Beer Bar,Bakery,Seafood Restaurant,Restaurant,Italian Restaurant,Farmers Market
1,Brockton / Parkdale Village / Exhibition Place,Café,Breakfast Spot,Coffee Shop,Bar,Bakery,Intersection,Italian Restaurant,Climbing Gym,Restaurant,Performing Arts Venue
2,Business reply mail Processing CentrE,Light Rail Station,Yoga Studio,Smoke Shop,Fast Food Restaurant,Farmers Market,Recording Studio,Auto Workshop,Spa,Restaurant,Pizza Place
3,CN Tower / King and Spadina / Railway Lands / ...,Airport Service,Airport Lounge,Airport Terminal,Plane,Sculpture Garden,Rental Car Location,Coffee Shop,Harbor / Marina,Boat or Ferry,Boutique
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Salad Place,Japanese Restaurant,Ice Cream Shop,Middle Eastern Restaurant,Burger Joint,Bubble Tea Shop


Cluster Analysis

In [15]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#toronto_grouped_clustering = Toronto_borough_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_borough_data = Toronto_borough_data.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_borough_data.head(1) # check the last columns!

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_borough_data['Latitude'],
                                  Toronto_borough_data['Longitude'],
                                  Toronto_borough_data['Neighborhood'],
                                  Toronto_borough_data['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters